In [ ]:
!pip install gymnasium[atari] --quiet
!pip install gymnasium --quiet
!pip install -U gymnasium[atari] --quiet
!pip install imageio_ffmpeg --quiet
!pip install npy_append_array --quiet
!pip install pyTelegramBotAPI --quiet
!pip install gymnasium[accept-rom-license] --quiet
!pip install gymnasium[box2d] --quiet

In [ ]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Conv2D, Input
 
class PolicyNetwork2D(tf.keras.Model):
    def __init__(self, action_dim=1):
        super(PolicyNetwork2D, self).__init__()
        self.fc1 = Dense(24, activation="relu")
        self.fc2 = Dense(36, activation="relu")
        self.fc3 = Dense(action_dim, activation="softmax")

    def call(self, state):
        x = self.fc1(state)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


In [ ]:
from tensorflow.keras.optimizers import Adam
import numpy as np 
import tensorflow_probability as tfp 

class PolicyGradientAgent: 
  
  def __init__(self, input_dims, out_dims, lr, action_space, gamma, chpkt, algo_name): 
    self.input_dims = input_dims
    self.out_dims = out_dims
    self.lr = lr
    self.action_space = action_space
    self.gamma = gamma
    self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
   
    self.policy_network = PolicyNetwork2D(self.out_dims)
    self.policy_network.compile(optimizer=Adam(learning_rate=self.lr))

    self.fname = chpkt + algo_name + "_"

  def save_model(self): 
    self.policy_network.save(self.fname)
    print("[+] Saved the model!!")

  def load_model(self):
    self.policy_network = tf.models.load_model(self.fname)
    print("[+] Loaded the mode!!")

  def policy(self, state): 
    state = state.reshape(1, -1)
    state = tf.convert_to_tensor(state)
    action_logits = self.policy_network(state)
    action = tf.random.categorical(tf.math.log(action_logits), num_samples=1)
    return action

  def get_action(self, state): 
    action = self.policy(state).numpy()
    return action.squeeze()
  
  def learn(self, rewards, actions, states): 
    discounted_rewards = []
    discounted_reward = 0

    for reward in rewards[::-1]: 
      discounted_reward = reward + self.gamma * discounted_reward 
      discounted_rewards.append(discounted_reward)

    discounted_rewards = discounted_rewards[::-1]

    for discounted_reward, state, action in zip(discounted_rewards, states, actions): 
     # discounted_rewards = tf.convert_to_tensor(discounted_rewards)
    #  states = tf.convert_to_tensor(states)
      #actions = tf.convert_to_tensor(actions)

      with tf.GradientTape() as tape: 
        action_probs = self.policy_network(np.array(state).reshape(1, -1), training=True)
        loss = self.loss(action_probs, action, discounted_reward)

      params = self.policy_network.trainable_variables
      grads = tape.gradient(loss, params)
      self.optimizer.apply_gradients(
                  zip(grads, params)
              ) 

  def loss(self, action_probabilities, actions, rewards): 
    dist = tfp.distributions.Categorical(
            probs=action_probabilities, dtype=tf.float32
        )
    log_prob = dist.log_prob(actions)
    loss = -log_prob * rewards
    return loss 
    

In [ ]:
import collections
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import tensorflow as tf
from gymnasium.wrappers import *


def manage_memory():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)


def plot_learning_curve(scores, figure_file):

    x = [_ for _ in range(len(scores))]
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    plt.savefig(figure_file)


def make_env(env_name): 
    env = gym.make(env_name, render_mode="rgb_array")
    
    if len(env.observation_space.shape) >= 3: 
        #env = AtariPreprocessing(env, 10, 4, 84, False, True)
        env = ResizeObservation(env, 84)
        env = GrayScaleObservation(env, keep_dim=False)
        env = FrameStack(env, 4, lz4_compress=False)
        env = NormalizeObservation(env)

    return env

In [ ]:
class Writer:
    def __init__(self, fname): 
        self.fname = fname 

    def write_to_file(self, content): 
        with open(self.fname, "a") as file: 
            file.write(content + "\n")

    def read_file(self, fname):
        with open(fname, "r") as file: 
            return file.read()
            

In [ ]:
import time
from telebot import TeleBot
import datetime
import telebot

token = "6238487424:AAG0jRhvbiVa90qUcf2fAirQr_-quPMs7cU"
chat_id = "1055055706"
bot = TeleBot(token=token) 

def telegram_send(message, bot):
    chat_id = "1055055706"
    bot.send_message(chat_id=chat_id, text=message)

def welcome_msg(multi_step, double_dqn, dueling):
    st = 'Hi! Starting learning with DQN Multi-step = %d, Double DQN = %r, Dueling DQN = %r' % (multi_step, double_dqn, dueling)
    telegram_send(st, bot)
    
def info_msg(episode, max_episode, reward, best_score, loss): 
    st = f"Current Episode: {episode}, Current Reward: {reward}, Max Episode: {max_episode}, Best Score: {best_score}, loss: {loss}"
    telegram_send(st, bot)

def end_msg(learning_time):
    st = 'Finished! Learning time: ' + str(datetime.timedelta(seconds=int(learning_time)))
    telegram_send(st, bot)
    print(st)


In [ ]:
import numpy as np 
import imageio


class RecordVideo: 
    
    def __init__(self, prefix_fname,  out_directory="videos/", fps=10): 
        self.prefix_fname = prefix_fname
        self.out_directory = out_directory
        self.fps = fps
        self.images = []
        
    def add_image(self, image): 
        self.images.append(image)
    
    def save(self, episode_no): 
        name = self.out_directory + self.prefix_fname + "_" + str(episode_no) + ".mp4"
        imageio.mimsave(name, [np.array(img) for i, img in enumerate(self.images)], fps=self.fps)
        self.images = []

In [ ]:
class Trainer: 
  def __init__(self, env, action_space, input_dims, out_dims, video_prefix, is_tg, noe, max_steps, record, lr, gamma, chkpt, algo_name): 
    self.env = env
    self.noe = noe 
    self.max_steps = max_steps 

    self.recorder = RecordVideo(video_prefix)
    self.is_tg = is_tg 
    self.record = record
    self.agent = PolicyGradientAgent(input_dims, out_dims, lr, action_space, gamma, chkpt, algo_name)
    
  def train(self): 

    ep_rewards = []
    avg_rewards = []
    best_reward = float("-inf")

    for episode in range(self.noe): 
      state = self.env.reset()
      rewards = []
      actions = []
      states = []

      if self.record and episode % 50 == 0: 
        img = self.env.render()
        self.recorder.add_image(img)

      for step in range(self.max_steps):
        
        if type(state) == tuple: 
          state = state[0]
        states.append(state) 
        action = self.agent.get_action(state)

        next_info = self.env.step(action)

        next_state, reward_prob, terminated, truncated, _ = next_info 
        done = terminated or truncated 

        rewards.append(reward_prob)
        actions.append(action)

        state = next_state

        if record and episode % 50 == 0:
          img = self.env.render()
          self.recorder.add_image(img)

        if done: 
          l = self.agent.learn(rewards, actions, states)
          break 
        
      if self.record and episode % 50 == 0:
        self.recorder.save(episode)

      if sum(rewards) > best_reward: 
        self.agent.save_model()
        best_reward = sum(rewards)

      ep_rewards.append(sum(rewards))
      avg_reward = np.mean(ep_rewards[-100:])
      avg_rewards.append(avg_reward)
      print(f"Episode: {episode} Reward: {sum(rewards)} Best Score: {best_reward}, Average Reward: {avg_reward}")

    return ep_rewards, avg_rewards



In [ ]:
import pickle 


env = make_env("CartPole-v1")
action_space = [_ for _ in range((env.action_space.n))]
n_actions = len(action_space)
input_dims = env.observation_space.shape
noe = 1000 
print(input_dims, n_actions)
max_steps = 100000000
video_prefix = "policy_gradient_reinforce"
is_tg = True 
record = True
lr = 1e-5
gamma = 0.94
chpkt = 'models/'
algo_name = "cartpole_reinforce"

if __name__ == "__main__": 
  trainer = Trainer(env, action_space, input_dims, n_actions, video_prefix, is_tg, noe, max_steps, record, lr, gamma, chpkt, algo_name)
  ep_rewards, avg_rewards = trainer.train()
  
  with open("pg_episode_rewards.obj", "wb") as f: 
    pickle.dump(ep_rewards, f)
  
  with open("pg_avg_rewards.obj", "wb") as f: 
    pickle.dump(avg_rewards, f)

(4,) 2


Episode: 0 Reward: 22.0 Best Score: 22.0, Average Reward: 22.0
Episode: 1 Reward: 15.0 Best Score: 22.0, Average Reward: 18.5
Episode: 2 Reward: 18.0 Best Score: 22.0, Average Reward: 18.333333333333332
Episode: 3 Reward: 13.0 Best Score: 22.0, Average Reward: 17.0
Episode: 4 Reward: 13.0 Best Score: 22.0, Average Reward: 16.2
Episode: 5 Reward: 17.0 Best Score: 22.0, Average Reward: 16.333333333333332
Episode: 6 Reward: 15.0 Best Score: 22.0, Average Reward: 16.142857142857142
Episode: 7 Reward: 13.0 Best Score: 22.0, Average Reward: 15.75
Episode: 8 Reward: 17.0 Best Score: 22.0, Average Reward: 15.88888888888889


Episode: 9 Reward: 33.0 Best Score: 33.0, Average Reward: 17.6
Episode: 10 Reward: 16.0 Best Score: 33.0, Average Reward: 17.454545454545453
Episode: 11 Reward: 19.0 Best Score: 33.0, Average Reward: 17.583333333333332


Episode: 12 Reward: 38.0 Best Score: 38.0, Average Reward: 19.153846153846153
Episode: 13 Reward: 22.0 Best Score: 38.0, Average Reward: 19.357142857142858
Episode: 14 Reward: 14.0 Best Score: 38.0, Average Reward: 19.0
Episode: 15 Reward: 25.0 Best Score: 38.0, Average Reward: 19.375
Episode: 16 Reward: 10.0 Best Score: 38.0, Average Reward: 18.823529411764707
Episode: 17 Reward: 12.0 Best Score: 38.0, Average Reward: 18.444444444444443


Episode: 18 Reward: 59.0 Best Score: 59.0, Average Reward: 20.57894736842105
Episode: 19 Reward: 29.0 Best Score: 59.0, Average Reward: 21.0
Episode: 20 Reward: 16.0 Best Score: 59.0, Average Reward: 20.761904761904763
Episode: 21 Reward: 40.0 Best Score: 59.0, Average Reward: 21.636363636363637
Episode: 22 Reward: 18.0 Best Score: 59.0, Average Reward: 21.47826086956522
Episode: 23 Reward: 15.0 Best Score: 59.0, Average Reward: 21.208333333333332
Episode: 24 Reward: 13.0 Best Score: 59.0, Average Reward: 20.88
Episode: 25 Reward: 20.0 Best Score: 59.0, Average Reward: 20.846153846153847
Episode: 26 Reward: 14.0 Best Score: 59.0, Average Reward: 20.59259259259259
Episode: 27 Reward: 38.0 Best Score: 59.0, Average Reward: 21.214285714285715
Episode: 28 Reward: 32.0 Best Score: 59.0, Average Reward: 21.586206896551722
Episode: 29 Reward: 36.0 Best Score: 59.0, Average Reward: 22.066666666666666
Episode: 30 Reward: 12.0 Best Score: 59.0, Average Reward: 21.741935483870968


...

Episode: 800 Reward: 223.0 Best Score: 357.0, Average Reward: 168.1
Episode: 801 Reward: 249.0 Best Score: 357.0, Average Reward: 170.47
Episode: 802 Reward: 228.0 Best Score: 357.0, Average Reward: 172.61
Episode: 803 Reward: 232.0 Best Score: 357.0, Average Reward: 172.96
Episode: 804 Reward: 249.0 Best Score: 357.0, Average Reward: 175.3
Episode: 805 Reward: 237.0 Best Score: 357.0, Average Reward: 176.69
Episode: 806 Reward: 238.0 Best Score: 357.0, Average Reward: 177.5
Episode: 807 Reward: 181.0 Best Score: 357.0, Average Reward: 179.12
Episode: 808 Reward: 214.0 Best Score: 357.0, Average Reward: 179.52
Episode: 809 Reward: 194.0 Best Score: 357.0, Average Reward: 180.09
Episode: 810 Reward: 122.0 Best Score: 357.0, Average Reward: 179.28
Episode: 811 Reward: 270.0 Best Score: 357.0, Average Reward: 180.7
Episode: 812 Reward: 103.0 Best Score: 357.0, Average Reward: 179.71
Episode: 813 Reward: 198.0 Best Score: 357.0, Average Reward: 181.53
Episode: 814 Reward: 125.0 Best Score:

Episode: 850 Reward: 82.0 Best Score: 357.0, Average Reward: 164.3
Episode: 851 Reward: 78.0 Best Score: 357.0, Average Reward: 162.93
Episode: 852 Reward: 105.0 Best Score: 357.0, Average Reward: 161.7
Episode: 853 Reward: 130.0 Best Score: 357.0, Average Reward: 160.77
Episode: 854 Reward: 82.0 Best Score: 357.0, Average Reward: 159.63
Episode: 855 Reward: 64.0 Best Score: 357.0, Average Reward: 158.28
Episode: 856 Reward: 78.0 Best Score: 357.0, Average Reward: 157.29
Episode: 857 Reward: 66.0 Best Score: 357.0, Average Reward: 155.72
Episode: 858 Reward: 110.0 Best Score: 357.0, Average Reward: 155.48
Episode: 859 Reward: 107.0 Best Score: 357.0, Average Reward: 154.15
Episode: 860 Reward: 65.0 Best Score: 357.0, Average Reward: 152.22
Episode: 861 Reward: 84.0 Best Score: 357.0, Average Reward: 150.73
Episode: 862 Reward: 73.0 Best Score: 357.0, Average Reward: 149.36
Episode: 863 Reward: 60.0 Best Score: 357.0, Average Reward: 148.37
Episode: 864 Reward: 77.0 Best Score: 357.0, A

Episode: 900 Reward: 178.0 Best Score: 357.0, Average Reward: 137.62
Episode: 901 Reward: 202.0 Best Score: 357.0, Average Reward: 137.15
Episode: 902 Reward: 211.0 Best Score: 357.0, Average Reward: 136.98
Episode: 903 Reward: 185.0 Best Score: 357.0, Average Reward: 136.51
Episode: 904 Reward: 165.0 Best Score: 357.0, Average Reward: 135.67
Episode: 905 Reward: 157.0 Best Score: 357.0, Average Reward: 134.87
Episode: 906 Reward: 159.0 Best Score: 357.0, Average Reward: 134.08
Episode: 907 Reward: 149.0 Best Score: 357.0, Average Reward: 133.76
Episode: 908 Reward: 190.0 Best Score: 357.0, Average Reward: 133.52
Episode: 909 Reward: 157.0 Best Score: 357.0, Average Reward: 133.15
Episode: 910 Reward: 169.0 Best Score: 357.0, Average Reward: 133.62
Episode: 911 Reward: 131.0 Best Score: 357.0, Average Reward: 132.23
Episode: 912 Reward: 117.0 Best Score: 357.0, Average Reward: 132.37
Episode: 913 Reward: 118.0 Best Score: 357.0, Average Reward: 131.57
Episode: 914 Reward: 159.0 Best Sc

Episode: 950 Reward: 180.0 Best Score: 357.0, Average Reward: 138.38
Episode: 951 Reward: 186.0 Best Score: 357.0, Average Reward: 139.46
Episode: 952 Reward: 190.0 Best Score: 357.0, Average Reward: 140.31
Episode: 953 Reward: 183.0 Best Score: 357.0, Average Reward: 140.84
Episode: 954 Reward: 206.0 Best Score: 357.0, Average Reward: 142.08
Episode: 955 Reward: 184.0 Best Score: 357.0, Average Reward: 143.28
Episode: 956 Reward: 220.0 Best Score: 357.0, Average Reward: 144.7
Episode: 957 Reward: 193.0 Best Score: 357.0, Average Reward: 145.97
Episode: 958 Reward: 204.0 Best Score: 357.0, Average Reward: 146.91
Episode: 959 Reward: 199.0 Best Score: 357.0, Average Reward: 147.83
Episode: 960 Reward: 195.0 Best Score: 357.0, Average Reward: 149.13
Episode: 961 Reward: 177.0 Best Score: 357.0, Average Reward: 150.06
Episode: 962 Reward: 163.0 Best Score: 357.0, Average Reward: 150.96
Episode: 963 Reward: 163.0 Best Score: 357.0, Average Reward: 151.99
Episode: 964 Reward: 182.0 Best Sco

1

In [ ]:
def greedy_policy(observation, model, action_space): 
    state = tf.convert_to_tensor([observation])
    actions = model(state)
    action = tf.math.argmax(actions, axis=1).numpy()[0]
    return action

In [ ]:
import random 
import imageio
import tensorflow as tf 

class Eval: 

    def __init__(self, env, model_path, number_of_episode=50):
        self.env = env 
        self.model = tf.keras.models.load_model(model_path)
        self.recorder = RecordVideo('dqn_lunarlander', 'test_videos/', 15)
        self.number_of_episode = number_of_episode
        
    def test(self): 
        rewards = []
        steps = []
        for episode in range(self.number_of_episode): 
            done = False
            reward = 0
            step = 0
            state = env.reset(seed=random.randint(0,500))
            if episode % 10 == 0: 
                img = env.render()
                self.recorder.add_image(img) 

            while not done:
                action =  greedy_policy(state, self.model, ACTION_SPACE)
                state, reward_prob, terminated, truncated, _ = env.step(action)
                done = terminated or truncated 
                reward += reward_prob
                step += 1 
                if episode % 10 == 0:
                    img = env.render()
                    self.recorder.add_image(img)
            
            rewards.append(reward)
            steps.append(step)
            recorder.save(1) if episode % 10 == 0
        
        return rewards, steps
